In [1]:
import ROOT as rt
from root_numpy import root2array, testdata, tree2array
import numpy as np
import pandas as pd

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib
matplotlib.use('Agg')
import scipy.ndimage as ndi

from tqdm import tqdm
import io
import os
import glob

Welcome to JupyROOT 6.24/06


In [3]:
# Parameters
fgname = "/eos/user/z/zwang/wcda/data/mc/g4s2_rec/gamma_rec_143800t.root"
fhname = "/eos/user/z/zwang/wcda/data/mc/g4s2_rec/hadron_rec_1970s.root"

In [4]:
# Tool function

def show_mem():
    import psutil
    "Show mem"
    info = psutil.virtual_memory()
    print( u'电脑总内存：%.4f GB' % (info.total / 1024 / 1024 / 1024) )
    print(u'当前使用的总内存占比：',info.percent)
    print(u'cpu个数：',psutil.cpu_count())

def flatten(items):
    "flatten anything"
    for item in items:
        if isinstance(item, (list, np.ndarray, tuple, np.void)):
            yield from flatten(item)
        else:
            yield item

In [11]:
# read root data
import re
fgamma=rt.TFile(fgname)
fhadron=rt.TFile(fhname)

tgamma = fgamma.Get("rec")
thadron = fhadron.Get("rec")

[str(i).split()[1] for i in list(tgamma.GetListOfBranches())]

['irun',
 'evt',
 'nhit',
 'nfit',
 'sump1',
 'sump2',
 'mjd',
 'sigma',
 'theta',
 'phi',
 'xc',
 'yc',
 'ra',
 'dec',
 'fee',
 'ch',
 'iconf',
 'npe',
 'npe_c',
 'dr',
 'tr',
 't',
 'x',
 'y',
 'cpx',
 'pinc',
 'csr',
 'dcore',
 'dangle',
 'dangle_real',
 'priE',
 'zenmc',
 'azimc',
 'xcmc',
 'ycmc',
 'h1',
 'weight',
 'dangle_real',
 'dcore_real']

In [ ]:

gpe = tree2array(tgamma, branches=['npe'])
gx = tree2array(tgamma, branches=['x'])
gy = tree2array(tgamma, branches=['y'])
gt = tree2array(tgamma, branches=['t'])
# gtr = tree2array(tgamma, branches=['tr'])
# giconf = tree2array(tgamma, branches=['iconf'])
# gtr = tree2array(tgamma, branches=['tr'])
# gdr = tree2array(tgamma, branches=['dr'])
# gcpx = tree2array(tgamma, branches=['cpx'])
# gpinc = tree2array(tgamma, branches=['pinc'])
# gE = tree2array(tgamma, branches=['priE'])
gnhit = tree2array(tgamma, branches=['nhit'])
# gfee = tree2array(tgamma, branches=['fee'])
# gch = tree2array(tgamma, branches=['ch'])
# gevt = tree2array(tgamma, branches=['evt'])
# gxcmc = tree2array(tgamma, branches=['xcmc'])
# gycmc = tree2array(tgamma, branches=['ycmc'])
# gzenmc = tree2array(tgamma, branches=['zenmc'])
# gazimc = tree2array(tgamma, branches=['azimc'])

hpe = tree2array(thadron, branches=['npe'])
hx = tree2array(thadron, branches=['x'])
hy = tree2array(thadron, branches=['y'])
ht = tree2array(thadron, branches=['t'])
# htr = tree2array(thadron, branches=['tr'])
# hiconf = tree2array(thadron, branches=['iconf'])
# htr = tree2array(thadron, branches=['tr'])
# hdr = tree2array(thadron, branches=['dr'])
# hcpx = tree2array(thadron, branches=['cpx'])
# hpinc = tree2array(thadron, branches=['pinc'])
# hE = tree2array(thadron, branches=['priE'])
hnhit = tree2array(thadron, branches=['nhit'])
# hfee = tree2array(thadron, branches=['fee'])
# hch = tree2array(thadron, branches=['ch'])
# hevt = tree2array(thadron, branches=['evt'])
# hxcmc = tree2array(thadron, branches=['xcmc'])
# hycmc = tree2array(thadron, branches=['ycmc'])
# hzenmc = tree2array(thadron, branches=['zenmc'])
# hazimc = tree2array(thadron, branches=['azimc'])
# np.savez("datag.npz", npe = gpe, x = gx, y = gy, t = gt, tr = gtr, iconf = giconf, dr = gdr, cpx = gcpx, pinc = gpinc, priE = gE, nhit = gnhit, fee = gfee, ch = gch, evt = gevt, xcmc = gxcmc, ycmc = gycmc, zenmc = gzenmc, azimc = gazimc)
# np.savez("datah.npz", npe = hpe, x = hx, y = hy, t = ht, tr = htr, iconf = hiconf, dr = hdr, cpx = hcpx, pinc = hpinc, priE = hE, nhit = hnhit, fee = hfee, ch = hch, evt = hevt, xcmc = hxcmc, ycmc = hycmc, zenmc = hzenmc, azimc = hazimc)

In [ ]:
detx = [e for e in tqdm(flatten(gx))]
dety = [e for e in tqdm(flatten(gy))]

In [ ]:
detx = np.unique(np.array(detx))
dety = np.unique(np.array(dety))
dety=dety[::-1]

In [ ]:
# See the grid
X, Y = np.meshgrid(detx, dety)
# for i in range(52):
plt.scatter(X, Y, alpha=0.5, s=10, c="b")

In [ ]:
# Data converter xy2ij

cxi = [[np.where(detx == x)[0][0] for x in gx[i][0]] for i in tqdm(range(len(gx)))]
cyj = [[np.where(dety == y)[0][0] for y in gy[i][0]] for i in tqdm(range(len(gy)))]
# cxi = [[np.where(detx == x)[0][0] for x in gx[i][0]] for i in tqdm(range(len(gx)))]
# cyj = [[np.where(dety == y)[0][0] for y in gy[i][0]] for i in tqdm(range(len(gy)))]

In [ ]:
# Data converter pe&t

cgpe = np.zeros((10000,52,120))+1
cgt = np.zeros((10000,52,120))+1
chpe = np.zeros((10000,52,120))+1
cht = np.zeros((10000,52,120))+1
for i in tqdm(range(1000)):
    for j in range(len(cxi[i])):
        cgpe[i][cyj[i][j]][cxi[i][j]] += gpe[i][0][j]
        cgt[i][cyj[i][j]][cxi[i][j]] += gt[i][0][j]
        chpe[i][cyj[i][j]][cxi[i][j]] += hpe[i][0][j]
        cht[i][cyj[i][j]][cxi[i][j]] += ht[i][0][j]

In [ ]:
#Target map

#Event id
ievent = 0
#Smoth sigma, 0 means no smooth
sigma = 0.5

#plt fig
plt.figure(dpi=100)
cm = plt.cm.get_cmap('magma')
plt.xticks(np.linspace(0,120,5),np.linspace(int(detx.min()),int(detx.max()),5))
plt.yticks(np.linspace(0,52,5),np.linspace(int(dety.max()),int(dety.min()),5))
plt.imshow(ndi.gaussian_filter(cgpe[ievent], sigma=sigma),aspect='auto',cmap=cm,interpolation='none',norm=colors.LogNorm(clip=True,vmin=cgpe[ievent].max()/50,vmax=cgpe[ievent].max())) #
plt.title("Nhit: %d" %(gnhit[ievent][0]))
# plt.contour(cgpe[ievent], [cgpe[ievent].max()-400, cgpe[ievent].max()-300, cgpe[ievent].max()-200])
plt.colorbar()

In [ ]:
# Data viewer

plt.figure()
cm = plt.cm.get_cmap('RdPu')
# for i in range(0):
for i in [ievent]:
    # plt.scatter(gx[i][0], gy[i][0], s=50, c=giconf[i][0], alpha=0.5, norm=colors.Normalize(min(giconf[i][0]),max(giconf[i][0])), cmap=cm)    #giconf
    # plt.scatter(gx[i][0], gy[i][0], s=50, c=gfee[i][0], alpha=0.5, norm=colors.Normalize(min(gfee[i][0]),max(gfee[i][0])), cmap=cm)          #gfee
    # plt.scatter(gx[i][0], gy[i][0], s=50, c=gch[i][0], alpha=0.5, norm=colors.Normalize(min(gch[i][0]),max(gch[i][0])), cmap=cm)             #gch
    plt.scatter(gx[i][0], gy[i][0], s=50, c=gt[i][0], alpha=0.5, norm=colors.Normalize(min(gt[i][0]),max(gt[i][0])), cmap=cm)                #gt
    # plt.scatter(gx[i][0], gy[i][0], s=50, c=gtr[i][0], alpha=0.5, norm=colors.Normalize(min(gtr[i][0]),max(gtr[i][0])), cmap=cm)             #gtr
    # plt.scatter(gx[i][0], gy[i][0], s=50, c=gpe[i][0], alpha=0.5, norm=colors.LogNorm(min(gpe[i][0]),max(gpe[i][0])), cmap=cm)               #gpe
    # plt.scatter(hx[i][0], hy[i][0], s=50, c=hpe[i][0], alpha=0.5, norm=colors.LogNorm(min(hpe[i][0]),max(hpe[i][0])), cmap=cm)               #hpe
plt.colorbar()
# ax=plt.gca()
# for i in range(gnhit[0][0]-200):
#     plt.text(gx[0][0][i], gy[0][0][i], giconf[0][0][i], fontsize=1, color = "r", transform=ax.transAxes, style = "italic", weight = "bold", verticalalignment='center', horizontalalignment='center')